<a href="https://colab.research.google.com/github/sankeawthong/Project-1-Lita-Chatbot/blob/main/Hybrid%20RF-Bi-LSTM%20for%204%20class%20classifications%20based%20on%20WSNBFSF%20dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **Hybrid RF-Bi-LSTM for 4 class classifications based on WSNBFSF dataset**

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Bidirectional, LSTM
from sklearn.metrics import classification_report
from imblearn.over_sampling import SMOTE

In [2]:
# Load the dataset
dataset = pd.read_csv("dataset.csv")
dataset.shape
dataset.isnull().sum()
dataset.info()
dataset["Class"].unique()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 312106 entries, 0 to 312105
Data columns (total 18 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   Event           312106 non-null  int64  
 1   Time            312106 non-null  float64
 2   S_Node          312106 non-null  int64  
 3   Node_id         312106 non-null  int64  
 4   Rest_Energy     312106 non-null  float64
 5   Trace_Level     312106 non-null  int64  
 6   Mac_Type_Pckt   312106 non-null  int64  
 7   Source_IP_Port  312106 non-null  float64
 8   Des_IP_Port     312106 non-null  float64
 9   Packet_Size     312106 non-null  int64  
 10  TTL             312106 non-null  int64  
 11  Hop_Count       312106 non-null  int64  
 12  Broadcast_ID    312106 non-null  int64  
 13  Dest_Node_Num   312106 non-null  int64  
 14  Dest_Seq_Num    312106 non-null  int64  
 15  Src_Node_ID     312106 non-null  int64  
 16  Src_Seq_Num     312106 non-null  int64  
 17  Class     

array([0, 1, 2, 3])

In [3]:
# Data preprocessing
y = dataset['Class'].values
X = dataset.drop(['Class'], axis=1)
X = X.values

In [4]:
# Data balancing using SMOTE
oversample = SMOTE()
X, y = oversample.fit_resample(X, y)

In [5]:
# Reshape the data for LSTM
X = X.reshape(X.shape[0], X.shape[1], 1)

In [6]:
# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [7]:
# Random Forest model
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train.reshape(X_train.shape[0], -1), y_train)  # Reshape X_train to 2D

y_pred_rf = rf_model.predict(X_test.reshape(X_test.shape[0], -1))  # Reshape X_test to 2D

from sklearn.metrics import accuracy_score

# Calculate accuracy for Random Forest model
accuracy_rf = accuracy_score(y_test, y_pred_rf)
print("Random Forest Accuracy:", accuracy_rf)

# Evaluate Random Forest model
rf_report = classification_report(y_test, y_pred_rf)
print("Random Forest Model Classification Report:")
print(rf_report)

Random Forest Accuracy: 0.9997241785991127
Random Forest Model Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     52705
           1       1.00      1.00      1.00     52517
           2       1.00      1.00      1.00     52703
           3       1.00      1.00      1.00     52356

    accuracy                           1.00    210281
   macro avg       1.00      1.00      1.00    210281
weighted avg       1.00      1.00      1.00    210281



In [8]:
from tensorflow.keras.utils import to_categorical
# Bi-LSTM model
lstm_model = Sequential()
lstm_model.add(Bidirectional(LSTM(64, return_sequences=True), input_shape=(X_train.shape[1], 1)))
lstm_model.add(Dropout(0.2))
lstm_model.add(Bidirectional(LSTM(64)))
lstm_model.add(Dropout(0.2))
lstm_model.add(Dense(4, activation='softmax'))
lstm_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
lstm_model.summary()

# Convert labels to one-hot encoded vectors
num_classes = len(np.unique(y))
y_train_encoded = to_categorical(y_train, num_classes)
y_test_encoded = to_categorical(y_test, num_classes)

lstm_model.fit(X_train, y_train_encoded, epochs=10, batch_size=64, validation_data=(X_test, y_test_encoded), verbose=1)

##y_pred_lstm = lstm_model.predict_classes(X_test)
y_pred_lstm_prob = lstm_model.predict(X_test)
y_pred_lstm = np.argmax(y_pred_lstm_prob, axis=1)

# Calculate accuracy for Bi-LSTM model
accuracy_lstm = accuracy_score(y_test, y_pred_lstm)
print("Bi-LSTM Accuracy:", accuracy_lstm)

# Evaluate Bi-LSTM model
lstm_report = classification_report(y_test, y_pred_lstm)
print("Bi-LSTM Model Classification Report:")
print(lstm_report)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bidirectional (Bidirectiona  (None, 17, 128)          33792     
 l)                                                              
                                                                 
 dropout (Dropout)           (None, 17, 128)           0         
                                                                 
 bidirectional_1 (Bidirectio  (None, 128)              98816     
 nal)                                                            
                                                                 
 dropout_1 (Dropout)         (None, 128)               0         
                                                                 
 dense (Dense)               (None, 4)                 516       
                                                                 
Total params: 133,124
Trainable params: 133,124
Non-trai

In [9]:
# Combine Random Forest and Bi-LSTM predictions
combined_pred = np.concatenate((y_pred_rf.reshape(-1, 1), y_pred_lstm.reshape(-1, 1)), axis=1)
y_pred_combined = [np.bincount(row).argmax() for row in combined_pred]

# Calculate accuracy for the combined model
accuracy_combined = accuracy_score(y_test, y_pred_combined)
print("Combined Model Accuracy:", accuracy_combined)

# Evaluate the combined model
combined_report = classification_report(y_test, y_pred_combined)
print("Combined Model Classification Report:")
print(combined_report)

Combined Model Accuracy: 0.9701161778762704
Combined Model Classification Report:
              precision    recall  f1-score   support

           0       0.92      1.00      0.96     52705
           1       0.98      0.91      0.94     52517
           2       0.99      1.00      0.99     52703
           3       1.00      0.97      0.99     52356

    accuracy                           0.97    210281
   macro avg       0.97      0.97      0.97    210281
weighted avg       0.97      0.97      0.97    210281

